In [11]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
from proj1_helpers import *
from data_helpers import *
from cross_validation import *
from plots import *

In [13]:
#LOAD TRAINING DATA
DATA_TRAIN_PATH = 'data/train.csv'
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

In [14]:
#CLEAN TRAINING DATA
bounds = [0.0, 0.3]
tx_clean, ind_remov = treat_undefined_values(bounds, tX)
ind_remov

[4, 5, 6, 12, 23, 24, 25, 26, 27, 28]

In [15]:
#MODEL BUILDING
tx, mean, std = standardize(tx_clean,0)
tx_augmented = build_poly_all_features(tx,4)
y,tx = build_model_data(tx_augmented,y)
y = classify(y)
num_samples = len(y)
num_features = tx.shape[1]
y = y.reshape(num_samples,1)


In [16]:
num_samples, num_features
tx.shape

(250000, 81)

In [17]:
from compute_gradient import *
from cost import *
from implementations import *

In [18]:
#Initial weight


#Maximum iterations through the whole set
max_iter = 100

#0.001 nan
#0.0005 nan

#Step-size
gammas = np.logspace(-6,-1, 7)
lambdas = np.logspace (-6,-1,7)


#Regularization factor
# lambda_ = 0.01 142000 loss 10 000 iters
#lambda_ = 0.001 141000 loss 10 000 iters
#lambda_ = 0.02
#loss_min = np.inf
tx.shape,y.shape,y

((250000, 81), (250000, 1), array([[1.],
        [0.],
        [0.],
        ...,
        [1.],
        [0.],
        [0.]]))

In [ ]:
losses_tr = np.zeros((len(gammas),len(lambdas)))
losses_val = np.zeros((len(gammas),len(lambdas)))

k_fold = 2
seed = 1
loss_min = np.inf
initial_w = np.zeros((tx.shape[1],1))
max_accuracy = 0
#CHOOSE LAMBDA AND GAMMA
for ind_lambda, lambda_ in enumerate(lambdas):
    for ind_gamma ,gamma_ in enumerate (gammas):
        loss_training = 0
        loss_validation = 0
        training_accuracy = 0
        validation_accuracy = 0
        for i,k in enumerate(range(k_fold)):
            
            tx_train, y_train, tx_val, y_val = cross_validation(y, tx, k, k_fold, seed)
            w, loss_train_k = reg_logistic_regression(y_train, tx_train, lambda_, initial_w, max_iter, gamma_)
            loss_val_k = reg_logistic_loss (y_val,tx_val,w,lambda_)/len(y_val)
            #loss_train_k /= len(y_train)
            loss_training += loss_train_k
            loss_validation += loss_val_k
            training_accuracy += predict_accuracy(y_train,tx_train,w)
            validation_accuracy += predict_accuracy(y_val,tx_val,w)
        
        training_accuracy /= k_fold
        validation_accuracy /= k_fold
        loss_training /= k_fold
        loss_validation /= k_fold
        print(lambda_)
        print(gamma_)
        print(loss_training)
        print(loss_validation)
        print(training_accuracy)
        print(validation_accuracy)
        if (validation_accuracy > max_accuracy):
            weights_star = w
            lambda_star = lambda_
            gamma_star = gamma_
            loss_min = loss_validation
            max_accuracy = validation_accuracy
        losses_tr[ind_lambda][ind_gamma] = loss_training
        losses_val[ind_lambda][ind_gamma] = loss_validation



/Users/Sofia/Desktop/ML_Project1/cost.py:20: RuntimeWarning: divide by zero encountered in log
  loss = y.T.dot(np.log(pred)) + (1 - y).T.dot(np.log(1 - pred))


1e-06
1e-06
0.6932239223408594
nan
65.7556
65.748
1e-06
6.8129206905796085e-06
0.6911682264584982
nan
65.854
65.8484
1e-06
4.641588833612782e-05
0.6680549000027372
nan
65.7284
65.73599999999999
1e-06
0.00031622776601683794
0.5777696647304253
nan
63.705200000000005
63.7284
1e-06
0.0021544346900318843
1.3135533218150246
nan
68.11840000000001
68.1704
1e-06
0.014677992676220705
1.0450052063438822
nan
67.76320000000001
67.674
1e-06
0.1
2.8549982756121652
nan
66.34
66.3708
6.8129206905796085e-06
1e-06
8.018804823159678
nan
66.34360000000001
66.3432
6.8129206905796085e-06
6.8129206905796085e-06
8.0185246864131
nan
66.3468
66.3464
6.8129206905796085e-06
4.641588833612782e-05
8.016614731286648
nan
66.3604
66.362
6.8129206905796085e-06
0.00031622776601683794
8.00350903990022
nan
66.4432
66.4444
6.8129206905796085e-06
0.0021544346900318843
7.905003766900641
nan
66.9724
66.946
6.8129206905796085e-06
0.014677992676220705
7.642846822016927
nan
66.6472
66.6576
6.8129206905796085e-06
0.1
4.13749676028

In [26]:
lambdas

array([1.00000000e-05, 4.64158883e-05, 2.15443469e-04, 1.00000000e-03,
       4.64158883e-03, 2.15443469e-02, 1.00000000e-01])

In [27]:
gammas

array([1.00000000e-06, 6.81292069e-06, 4.64158883e-05, 3.16227766e-04,
       2.15443469e-03, 1.46779927e-02, 1.00000000e-01])

In [28]:
losses_tr, losses_val

(array([[0.69319692,        inf,        nan,        nan,        nan,
                nan,        nan],
        [       nan,        nan,        nan,        nan,        nan,
                nan,        nan],
        [       nan,        nan,        nan,        nan,        nan,
                nan,        nan],
        [       nan,        nan,        nan,        nan,        nan,
                nan,        nan],
        [       nan,        nan,        nan,        nan,        nan,
                nan,        nan],
        [       nan,        nan,        nan,        nan,        nan,
                nan,        nan],
        [       nan,        nan,        nan,        nan,        nan,
                nan,        nan]]),
 array([[0.69288124, 0.69210063,        nan,        nan,        nan,
                nan,        nan],
        [       nan,        nan,        nan,        nan,        nan,
                nan,        nan],
        [       nan,        nan,        nan,        nan,        nan,
  

In [29]:
loss_min, lambda_star, gamma_star

(0.6921006315995246, 1e-05, 6.8129206905796085e-06)

In [10]:
#PREDICT TRAINING ACCURACY
training_accuracy = predict_accuracy(y,tx,weights_star)
training_accuracy

NameError: name 'weights_star' is not defined

In [198]:
#LOAD TEST SET
DATA_TEST_PATH = 'data/test.csv'
ytest, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [203]:
#CLEAN AND STANDARDIZE TEST SET
tX_test_clean = np.delete(tX_test, ind_remov, axis=1)
tX_test_clean.shape


(568238, 23)

In [206]:
tx_test_clean = remove_undefined_values (tX_test_clean)
tx_test_clean.shape

(568238, 23)

In [207]:
tx_test_clean, _, _ = standardize(tx_test_clean,0)


In [210]:
tx_test_augmented = build_poly_all_features(tx_test_clean,4)

(568238, 23)

In [213]:
tx_test_augmented.shape

(568238, 46)

In [214]:
#BUILD TEST MODEL
ytest, tx_test = build_model_data(tx_test_augmented,ytest)

In [215]:
#PREDICT LABELS
OUTPUT_PATH = 'data/submission_reg_logistic-degre3.csv'
y_pred = predict_labels(weights_star, tx_test,'logistic')
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)